In [2]:
import pandas as pd
import numpy as np

In [3]:
trip = pd.read_csv('data/tripP')
train = pd.read_csv('data/trip_train.csv', usecols=['id'])
test = pd.read_csv('data/trip_test.csv', usecols=['id'])

/home/pabloernesto/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train = train.merge(trip, on='id', how='left')
test = test.merge(trip, on='id', how='left')

In [5]:
train.head(1)

,id,duration,date,start_station_id,subscription_type,time,year,month,day,dayofweek,date.1,holiday
0,907649,396,2015-08-27,50,1,516,2015,8,27,4,NaN,NaN


In [6]:
from sklearn import preprocessing
cols = ['start_station_id', 'subscription_type', 'time', 'year', 'month', 'day','dayofweek']
X = train[cols].apply(lambda x: preprocessing.MinMaxScaler().fit_transform(x), axis=0)
y = train.duration

/home/pabloernesto/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
/home/pabloernesto/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/pabloernesto/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/h

In [7]:
X_test = test[cols].apply(lambda x: preprocessing.MinMaxScaler().fit_transform(x), axis=0)
y_test = test.duration

/home/pabloernesto/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/pabloernesto/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/pabloernesto/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.re

In [8]:
X.sample(5)

,start_station_id,subscription_type,time,year,month,day,dayofweek
140691,0.890244,1.0,0.478805,0.0,0.909091,0.666667,0.500000
330863,0.646341,1.0,0.505907,1.0,0.272727,0.433333,0.166667
13301,0.914634,1.0,0.673384,1.0,0.545455,0.866667,0.000000
409811,0.829268,1.0,0.271022,1.0,0.454545,0.033333,0.166667
336196,0.756098,1.0,0.708131,0.5,0.818182,0.766667,0.666667


In [15]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import FeatureAgglomeration
#clustering = KMeans(n_clusters=11,n_jobs=-1)
#clustering = DBSCAN(n_jobs=-1)
clustering = FeatureAgglomeration(n_clusters=7)

In [19]:
clustering = clustering.fit(X)
#X['cluster'] = clustering.transform(X)
#X_test['cluster'] = clustering.transform(X_test)
clustering.transform(X).shape

(549961, 7)

In [1]:
X.sample(3)

NameError: name 'X' is not defined

In [ ]:
X_test.sample(3)

In [19]:
from sklearn import neural_network
pred = neural_network.MLPRegressor().fit(X, y).predict(X_test)

In [20]:
from sklearn.metrics import mean_squared_error
mean_squared_error(pred, y_test)

41546893.938332714

In [23]:
predictionDF = pd.DataFrame(pred,columns={"duration"})

In [24]:
predictionDF.loc[:,'id'] = test['id']

In [25]:
predictionDF.to_csv(path_or_buf='data/neural_networks_with_clustering',sep=',',header=True,columns=['id','duration'],index=False)